In [64]:
from google.cloud import storage
import pandas as pd
import io
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [65]:
def load_data_from_gcs(bucket_name, file_path):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    content = blob.download_as_bytes()
    return pd.read_csv(io.BytesIO(content))

In [80]:
bucket_name = "ml_databases"

negocios=load_data_from_gcs(bucket_name, "negocios.csv")
reviews=load_data_from_gcs(bucket_name, "reviews.csv")
ciudades=load_data_from_gcs(bucket_name, "ciudades.csv")
atributos=load_data_from_gcs(bucket_name, "atributos.csv")
categorias=load_data_from_gcs(bucket_name, "categorias.csv")
negocios_categorias=load_data_from_gcs(bucket_name, "negocios_categorias.csv")
negocios_atributos=load_data_from_gcs(bucket_name, "negocios_atributos.csv")
categorias=categorias.dropna(subset="category")

In [81]:


# Unir negocios con ciudades
negocios_ciudades = negocios.merge(ciudades, on="city_id", how="left")

# Unir negocios con categorías
negocios_categorias = negocios_categorias.merge(categorias, on="category_id", how="left")
negocios_categorias = negocios_categorias.groupby("id")["category"].apply(lambda x: ", ".join(x)).reset_index()

# Unir negocios con atributos
negocios_atributos = negocios_atributos.merge(atributos, on="atributo_id", how="left")
negocios_atributos = negocios_atributos.groupby("id")["atributo"].apply(lambda x: ", ".join(x)).reset_index()

# Unir negocios con reviews (obteniendo el promedio de rating y sentimiento)
reviews_agg = reviews.groupby("id").agg(
    avg_rating=("rating", "mean"),
    avg_vader_score=("vader_score", "mean"),
    avg_textblob_score=("textblob_score", "mean"),
    review_count=("rating", "count")
).reset_index()

# Unir todo en un solo dataset
negocios_final = negocios_ciudades.merge(negocios_categorias, on="id", how="left") \
                                        .merge(negocios_atributos, on="id", how="left") \
                                        .merge(reviews_agg, on="id", how="left")

# Separar las categorías individuales y contar cada una por ciudad correctamente
negocios_categorias_expandidas = negocios_final.copy()
negocios_categorias_expandidas = negocios_categorias_expandidas.assign(
    category=negocios_categorias_expandidas["category"].str.split(", ")
).explode("category")

# Recalcular la competencia correctamente (negocios por categoría en cada ciudad)
competencia = negocios_categorias_expandidas.groupby(["city", "category"]).size().reset_index(name="competencia")

# Crear tabla ciudad-categoría con métricas relevantes
ciudad_categoria = negocios_categorias_expandidas.groupby(["city", "category"]).agg(
    avg_rating=("avg_rating", "mean"),  
    avg_vader_score=("avg_vader_score", "mean"),  
    avg_textblob_score=("avg_textblob_score", "mean"),  
    poblacion=("population", "first")  
).reset_index()

# Unir con la competencia corregida
ciudad_categoria = ciudad_categoria.merge(competencia, on=["city", "category"], how="left")

# Manejar valores nulos en métricas clave
ciudad_categoria.fillna({'competencia': 0, 'avg_rating': 3.0, 'avg_vader_score': 0.0, 'avg_textblob_score': 0.0}, inplace=True)

# Nueva definición de "recomendable" con lógica mejorada
ciudad_categoria["competencia_relativa"] = ciudad_categoria.groupby("city")["competencia"].transform(
    lambda x: (x - x.mean()) / x.std()
)

ciudad_categoria["recomendable"] = (
    (ciudad_categoria["competencia_relativa"] < -0.5) &  # Competencia baja en comparación con la media de la ciudad
    (ciudad_categoria["avg_rating"] < 3.5) &  # Baja calificación
    (ciudad_categoria["avg_vader_score"] < 0.2) &  # Sentimiento negativo en VADER
    (ciudad_categoria["avg_textblob_score"] < 0.2)  # Sentimiento negativo en TextBlob
).astype(int)

# Guardar el dataset procesado
ciudad_categoria.to_csv("ciudad_categoria_procesado.csv", index=False)

# Confirmación de que los datos están listos
ciudad_categoria.head()


,city,category,avg_rating,avg_vader_score,avg_textblob_score,poblacion,competencia,competencia_relativa,recomendable
0,Alachua,Bars & Nightlife,4.613861,0.423762,0.298653,10770,1,-0.866025,0
1,Alachua,Chinese,3.000000,0.000000,0.000000,10770,2,0.866025,0
2,Alachua,Eyebrow & Beauty Bars,3.000000,0.000000,0.000000,10770,1,-0.866025,1
3,Alachua,Restaurant,3.666667,0.111111,0.105944,10770,2,0.866025,0
4,Alafaya,Restaurant,2.833333,-0.041917,0.092083,90874,1,NaN,0


In [83]:
#Modelo Machile Learning
# Seleccionar características para el modelo
features = ["competencia", "avg_rating", "avg_vader_score", "avg_textblob_score", "poblacion"]
target = "recomendable"

# Manejar valores nulos en las nuevas variables
ciudad_categoria["avg_vader_score"] = ciudad_categoria["avg_vader_score"].fillna(0)
ciudad_categoria["avg_textblob_score"] = ciudad_categoria["avg_textblob_score"].fillna(0)

# Separar variables predictoras (X) y variable objetivo (y)
X = ciudad_categoria[features]
y = ciudad_categoria[target]

# Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo XGBoost con las variables separadas
modelo_xgb = XGBClassifier(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

modelo_xgb.fit(X_train, y_train)

#Guardar Modelo Entrenado
joblib.dump(modelo_xgb, "modelo_xgb_1.pkl")

# Hacer predicciones en el conjunto de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Evaluar el modelo con las nuevas características
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
report_xgb = classification_report(y_test, y_pred_xgb)

# Mostrar la precisión y el reporte de clasificación
print("Precisión del modelo XGBoost optimizado:", accuracy_xgb)
print("Reporte de Clasificación:\n", report_xgb)

Precisión del modelo XGBoost optimizado: 0.9379715004191115
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       914
           1       0.86      0.88      0.87       279

    accuracy                           0.94      1193
   macro avg       0.91      0.92      0.91      1193
weighted avg       0.94      0.94      0.94      1193

